# Import libraries

In [1]:
import sigma
from sigma.utils import normalisation as norm 
from sigma.utils import visualisation as visual
from sigma.utils.loadtem import TEMDataset
from sigma.src.utils import same_seeds
from sigma.src.dim_reduction import Experiment
from sigma.models.autoencoder import AutoEncoder
from sigma.src.segmentation import PixelSegmenter
from sigma.gui import gui


# Load emi/ser files
Note that the `.emi` and the corresponding `.ser` files should be place in the same folder, e.g., `test.emi` and `test.ser` should be in the same folder.

In [3]:
file_path = '/Users/andrewtung/Dropbox (Cambridge University)/MyFolder/Github/sigma/tutorial/17.57.40 Spectrum image.emi' # file path
tem = TEMDataset(file_path)

# Assign x-ray lines (if known)

In [4]:
tem.set_xray_lines(['C_Ka', 'O_Ka', 'Mg_Ka', 'Al_Ka', 
                    'Si_Ka', 'Ti_Ka','S_Ka', 'Ca_Ka',  
                    'Fe_Ka', 'Ni_Ka'])

Set xray_lines to ['C_Ka', 'O_Ka', 'Mg_Ka', 'Al_Ka', 'Si_Ka', 'Ti_Ka', 'S_Ka', 'Ca_Ka', 'Fe_Ka', 'Ni_Ka']


# View EDX dataset and assign x-ray lines

In [ ]:
gui.view_emi_dataset(tem)

In [5]:
# normalisation to make the spectrum of each pixel summing to 1.
tem.peak_intensity_normalisation()

# remove pixels with no signal
tem.remove_NaN()


Normalising the chemical intensity along axis=2, so that the sum is wqual to 1 along axis=2.


In [6]:
tem.normalisation([norm.zscore, 
                   norm.softmax])

Set feature_list to ['C_Ka', 'O_Ka', 'Mg_Ka', 'Al_Ka', 'Si_Ka', 'Ti_Ka', 'S_Ka', 'Ca_Ka', 'Fe_Ka', 'Ni_Ka']
Normalise dataset using:
    1. zscore
    2. softmax


In [7]:
# The integer in this function can determine different initialised parameters of model (tuning sudo randomness)
# This can influence the result of dimensionality reduction and change the latent space.
same_seeds(1)

# Set up the experiment, e.g. determining the model structure, dataset for training etc.
general_results_dir='./' 
ex = Experiment(descriptor='softmax',
                general_results_dir=general_results_dir,
                model=AutoEncoder,
                model_args={'hidden_layer_sizes':(512,256,128)}, # number of hidden layers and corresponding neurons
                chosen_dataset=tem.normalised_elemental_data,
                save_model_every_epoch=True)

model_name: Model-softmax
size_dataset: (173, 256, 10)
device: cpu
num_parameters: 343820


In [8]:
# Train the model
ex.run_model(num_epochs=5,
             patience=50, 
             batch_size=64,
             learning_rate=1e-4, 
             weight_decay=0.0, 
             task='train_all', # Change to 'train_eval' to train on the training set (85% dataset) and test on a testing set (15%) for evaluation
             noise_added=0.0,
             KLD_lambda=0.0,
             criterion='MSE',
             lr_scheduler_args={'factor':0.5,
                                'patience':5, 
                                'threshold':1e-2, 
                                'min_lr':1e-6,
                                'verbose':True}) 

num_epochs: 5
batch_size: 64
task: train_all
optimizer: lr=0.0001 and weight_decay=0.0

Start training ...



  0%|          | 0/692 [00:00<?, ?batch/s]

Epoch 1 ----> model saved, train_loss=0.001996 | test_loss = 0.001996


  0%|          | 0/692 [00:00<?, ?batch/s]

Epoch 2 ----> model saved, train_loss=0.000814 | test_loss = 0.000814


...

Epoch 3 ----> model saved, train_loss=0.000715 | test_loss = 0.000715


  0%|          | 0/692 [00:00<?, ?batch/s]

Epoch 4 ----> model saved, train_loss=0.000661 | test_loss = 0.000661


  0%|          | 0/692 [00:00<?, ?batch/s]

Epoch 5 ----> model saved, train_loss=0.000637 | test_loss = 0.000637


In [10]:
latent = ex.get_latent()
ps = PixelSegmenter(latent, 
                    tem.normalised_elemental_data, 
                    tem,
                    method_args={'n_components':20, 'random_state':6, 'init_params':'kmeans'} )
                    # can change random_state to different integer i.e. 10 or 0 to adjust the clustering result.

: 

: 

In [9]:
# Plot latent sapce (2-dimensional) with corresponding Gaussian models
gui.view_latent_space(ps, color=True)

NameError: name 'ps' is not defined

In [ ]:
# visualise the latent space
gui.check_latent_space(ps,ratio_to_be_shown=0.5, show_map=True)

In [ ]:
# check the density of latent space
gui.plot_latent_density(ps, bins=50)

In [ ]:
gui.show_cluster_distribution(ps)

In [ ]:
gui.view_phase_map(ps)

In [ ]:
gui.view_clusters_sum_spectra(ps, normalisation=True, spectra_range=(0,8))

In [ ]:
weights, components = ps.get_unmixed_edx_profile(clusters_to_be_calculated='All', 
                                                 n_components='All',
                                                 normalised=False, 
                                                 method='NMF', 
                                                 method_args={'init':'nndsvd'})

In [ ]:
gui.show_unmixed_weights_and_compoments(ps, weights, components)